In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm import tqdm
from os import getcwd, path as base_path
from spacy import displacy
from collections import Counter
import en_core_web_sm
from typing import List
import numpy as np

import io
import re

In [3]:
from sys import path
PROJECT_PATH = getcwd()[:getcwd().find("notebooks")][:-1]
path.append(f"{PROJECT_PATH}\\src")

In [4]:
from utils import (get_words_contractions, 
                   find_urls_in_text, 
                   get_stopwords)

In [5]:
PROJECT_NAME = "wsj_tweet_scrapping"
PROJECT_PATH = getcwd()[: getcwd().find(PROJECT_NAME) + len(PROJECT_NAME)]
DATA_PATH = f"{PROJECT_PATH}\\data"
EXCEL_PATH = f"{DATA_PATH}\\excel"

In [15]:
from_year = 2011
to_year = 2020
news_df = pd.read_excel(f"{EXCEL_PATH}\\wsj_logistics_news_cleaned_{from_year}-{to_year}.xlsx")
news_df.dropna(inplace=True)

In [16]:
news_df.head()

,date,tweet,url,news,news_cleaned
0,2020-09-24,Today’s newsletter - Cleaning Vehicle Emissi...,https://on.wsj.com/2EuJDdb,\n\tNews and analysis on the world of logisti...,California is taking an aggressive step to ove...
1,2020-09-16,FedEx's Christmas in July; Probing Nikola Cl...,https://on.wsj.com/2Fsewzz,\n\tNews and analysis on the world of logisti...,FedEx is turning the surge in e-commerce deman...
2,2020-09-14,Shipping’s E-Commerce Drive; Short-Selling N...,https://on.wsj.com/3iv2qDW,\n\tNews and analysis on the world of logisti...,Container shipping lines are increasingly tryi...
3,2020-09-10,Brexit’s New Alarms; Railroad Offloads Bidde...,https://on.wsj.com/3inRU1m,\n\tNews and analysis on the world of logisti...,Fears over the fallout from a “hard” Brexit ar...
4,2020-09-23,Airlines Turn to Freight; Driving Robot Truc...,https://on.wsj.com/32TvO1a,\n\tNews and analysis on the world of logisti...,Just four of the world’s 30 largest passenger ...


# Name Entity Recognitin from spaCy

In [17]:
nlp = en_core_web_sm.load()

In [18]:
def identify_label(text : str, label_type: List):
    """Return list of companies identified"""
    label_text = []
    for x in nlp(text).ents:
        if x.label_ in label_type:
            label_text.append(x.text)
    return list(np.unique(label_text))

In [19]:
ORGS = []
for i in tqdm(range(len(news_df))):
    companies = identify_label(news_df.iloc[i]["news_cleaned"], label_type=["ORG"])
    ORGS.append(companies)

100%|██████████████████████████████████████████████████████████████████████████████| 2473/2473 [07:15<00:00,  5.68it/s]


In [24]:
news_df["companies_identified"] = ORGS

In [25]:
news_df.head()

,date,tweet,url,news,news_cleaned,companies_identified
0,2020-09-24,Today’s newsletter - Cleaning Vehicle Emissi...,https://on.wsj.com/2EuJDdb,\n\tNews and analysis on the world of logisti...,California is taking an aggressive step to ove...,"[Alejandro Lazo, Amazon, Bpost’s North America..."
1,2020-09-16,FedEx's Christmas in July; Probing Nikola Cl...,https://on.wsj.com/2Fsewzz,\n\tNews and analysis on the world of logisti...,FedEx is turning the surge in e-commerce deman...,"[A.P. Moller-Maersk, Asda, D.G. Yuengling & So..."
2,2020-09-14,Shipping’s E-Commerce Drive; Short-Selling N...,https://on.wsj.com/3iv2qDW,\n\tNews and analysis on the world of logisti...,Container shipping lines are increasingly tryi...,"[Amazon, Arm Holdings, CMA CGM, European Union..."
3,2020-09-10,Brexit’s New Alarms; Railroad Offloads Bidde...,https://on.wsj.com/3inRU1m,\n\tNews and analysis on the world of logisti...,Fears over the fallout from a “hard” Brexit ar...,"[Air Transport Services Group, Albany Times-Un..."
4,2020-09-23,Airlines Turn to Freight; Driving Robot Truc...,https://on.wsj.com/32TvO1a,\n\tNews and analysis on the world of logisti...,Just four of the world’s 30 largest passenger ...,"[Asiana Airlines, Beverage Daily, Brookfield, ..."


In [26]:
news_df.to_excel(f"{EXCEL_PATH}\\wsj_logistics_news_companies_identified_{from_year}-{to_year}.xlsx",
                 index=False)

In [27]:
news_df.iloc[0]["news_cleaned"]

"California is taking an aggressive step to overhaul driving in the state that could send auto makers and truck operators scrambling. An executive order signed by Gov. Gavin Newsom aims to end the sale of new gasoline and diesel-powered passenger cars in the state by 2035. The WSJ’s Alejandro Lazo and Russell Gold report the California Air Resources Board will be charged with developing the specific regulations to implement the state mandate for passenger cars and trucks. The order on trucks is less ambitious, calling for medium- and heavy-duty vehicles such as trucks to be zero emission by 2045 “where feasible.” New trucks serving ports would have to meet the 2035 deadline, however. Port trucking has been a flashpoint for complaints over freight-transport emissions, and a handful of electric or hybrid trucks have been used in demonstration projects at the state’s big maritime gateways. A prototype truck at an event hosted by Iveco and Nikola. Nikola’s investor-related distress is star